In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
# load the data
df = pd.read_csv('data/churn.csv')

# showing the shape (row, col) and  first 5 rows
print(df.shape)
df.head()

NameError: name 'pd' is not defined

In [ ]:
# removing columns that 'add no value': CustomerId, Surname
# these don't help prediction
df = df.drop(['CustomerId', 'Surname'], axis=1)

# checking the shape (row, col)
print(df.shape)
df.head()

In [ ]:
# checking for null values
# if 0 - no missing data
# if value > 0 - we have missnig data
df.isnull().sum()